# 激活函数（加入非线性的因素，解决线性模型缺陷）

## （一）sigmoid
- 将输入值压缩到0-1之间，但是在极大和极小的时候，会趋于饱和。饱和的意思是：x=100和x=1000时反映都是一样的，通过sigmoid的特征转换，可能丢失了信息
- 当特征间相差不是特别大，需要细微的分类判别时，效果较好

## （二）Tanh
- 同样也存在饱和问题，只是值域变成了-1-1
- 当特征相差明显时，在循环过程中将扩大特征效果并显示出来

## （三）Relu
- 与人类细胞类似，单边抑制，提升了机器的效率
- 但同时，由于对负值全部舍去，很容易导致模型输出全为0然后无法进行训练
- 数据有稀疏性，保留数据特征，用稀疏数据表示原数据

# softmax算法——分类问题

## （一）Softmax是什么


# 损失函数——真实值和预测值的距离来指导模型收敛方向

## 1.均方误差（MSE）


## 2.交叉熵
- 计算loss需要注意的点
    - tf.nn.softmax_cross_entropy_with_logits(logits, labels, name=None)
        - 第一个参数logits：就是神经网络最后一层的输出，如果有batch的话，它的大小就是[batchsize，num_classes]，单样本的话，大小就是num_classes
        - 第二个参数labels：实际的标签，大小同上
    - 
      \begin{equation}
        Softmax{(x)_i} = \frac{e^{x_i}}{\sum_j e^{x_j}}
        \label{eq:Softmax函数}
      \end{equation}

# 综合应用实验

## 1.交叉熵实验

In [1]:
import tensorflow as tf

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# ground_truth值，
labels = [[0,0,1],[0,1,0]]
# 网络输出值
logits = [[2,0.5,6],[0.1,0,3]]

- （1）将输出值logits分别进行两次softmax，看看有啥区别呢？

In [3]:
logits_scaled = tf.nn.softmax(logits)
logits_scaled2 = tf.nn.softmax(logits_scaled)

- (2) 观察交叉熵，将步骤（1）中的两个值分别进行softmax_cross_entropy_with_logits,观察区别

In [4]:
result1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits)
result2 = tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits_scaled)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



- (3)自建公式

In [5]:
result3 = -tf.reduce_sum(labels*tf.log(logits_scaled),1)  # reduce_mean和reduce_sum哪个能替代softmax的交叉熵函数

In [6]:
with tf.Session() as sess:
    print('Scaled:',sess.run(logits_scaled))  
    print('Scaled2:',sess.run(logits_scaled2))
    print('result1:',sess.run(result1))  # 正确的结果
    print('result2:',sess.run(result2))
    print('result3:',sess.run(result3))

Scaled: [[0.01791432 0.00399722 0.97808844]
 [0.04980332 0.04506391 0.90513283]]
Scaled2: [[0.21747023 0.21446465 0.56806517]
 [0.2300214  0.22893383 0.5410447 ]]
result1: [0.02215516 3.0996735 ]
result2: [0.56551915 1.4743223 ]
result3: [0.02215518 3.0996735 ]


#### 结果分析
- (1)两次softmax结果对比
    - 根据数据假设，logits的结果第一个是和标签相符的，第二个和标签不符，所以第一个的交叉熵较小，第二个的交叉熵较大
    - 经过第二次softmax后，分布概率有所变化
- (2)自建与公式实现tf.nn.softmax_cross_entropy_with_logits的效果,两者等价
    - result1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits)
    - result3 = -tf.reduce_sum(labels*tf.log(logits_scaled),1) 

### 2.one_hot实验
- 对非one-hot编码为标签的数据进行交叉熵的计算，比较其与one-hot编码的交叉熵之间的差别

In [7]:
# 标签总概率为1
labels = [[0.4,0.1,0.5],[0.3,0.6,0.1]]
result4 = tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits)
with tf.Session() as sess:
    print('result4:',sess.run(result4))

result4: [2.1721554 2.7696736]


#### 结果分析
- 对于非one-hot的标签，正确分类的交叉熵和错误分类的交叉熵差别没有one-hot那么明显

### 3.Sparse交叉熵的使用
- 可以直接用非one-hot的标签

In [8]:
# sparse标签
labels = [2,1] # 说明label中总共有3类：0,1,2
result5 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logits)
with tf.Session() as sess:
    print('result5:',sess.run(result5))

result5: [0.02215516 3.0996735 ]


### 4.计算loss值

In [9]:
labels = [[0,0,1],[0,1,0]]
loss1 = -tf.reduce_mean(tf.reduce_sum(labels*tf.log(logits_scaled),1))
loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits))
with tf.Session() as sess:
    print('loss1=',sess.run(loss1))
    print('loss2=',sess.run(loss2))

loss1= 1.5609144
loss2= 1.5609143


# 梯度下降——让模型逼近最小偏差 
## 常用方法：

### 1. 批量梯度下降
- 遍历全部数据集计算一次损失函数，然后更新参数。每更新一次参数就要把数据集中的所有样本看一遍，计算量大，速度慢

### 2.随机梯度下降
- 每看一个数据就计算一次损失函数，然后求梯度更新参数。速度较快，但是收敛性不是很好，两次参数更新很有可能互相抵消，造成目标函数震荡剧烈

### 3.小批量梯度下降
- 折中的手段，将数据分为若干批次，按照批次来更新参数，每一批中的一组数据共同决定本次梯度的方向，下降起来就不容易跑偏，减少了随机性；然后每批次的量又不像批量梯度下降那么大，降低了计算量。Bingo

## 学习率衰减
- 学习率比较大的时候训练速度提升，但是精度不够，较小的时候精度提升但是训练的慢，所以引入退化学习率，刚开始的时候使用大的学习率加快速度，训练到一定程度的时候利用小的学习率来提高精度

## 实验：退化学习率的用法举例

In [1]:
import tensorflow as tf
global_step =  tf.Variable(0,trainable=False)
initial_learning_rate = 0.1
learning_rate = tf.train.exponential_decay(initial_learning_rate,global_step,decay_steps=10,decay_rate=0.9)
opt = tf.train.GradientDescentOptimizer(learning_rate)
add_global = global_step.assign_add(1)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(learning_rate))
    for i in range(10):
        g,rate = sess.run([add_global,learning_rate])
        print(g,rate)

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0.1
1 0.1
2 0.09895193
3 0.09791484
4 0.095873155
5 0.094868325
6 0.094868325
7 0.09387404
8 0.092890166
9 0.09095325
10 0.09095325
